In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import datetime
import time

opt =  webdriver.ChromeOptions()
opt.add_argument('--start-maximized')
opt.add_argument('--disable-extensions')


driver_path = r'webdriver/chromedriver'
driver = webdriver.Chrome(options=opt)

# inicializar navegador y cargar página completa, espefificar día y hora
date_time = str(datetime.datetime.now())

import pandas as pd
productos = pd.read_csv('productos.csv')


# funcion de scrapping
def scrap_page(url,art,store):
    store=store
    driver.get(url)
    time.sleep(5)
    print("se busca en",url,"en fecha y hora:",date_time)
    art=art


    # scrapear data scaneando todas las páginas (especificar rango)
    if store=='megatone':
        itemlist = []
        data=[]
        for i in range (100):
            page=i+2
            items = driver.find_elements(By.CSS_SELECTOR,'div.w100.dIB.AnchoInterno.TextDeco.pR')

            for i in range(len(items)):
                itemlist.append(items[i].text)
            itemlist = list(filter(None,itemlist))
            arrayitems=[]
            for i in range (len(itemlist)):
        
                words=str(itemlist[i]).split('\n')
                words=[x for x in words if 'Sin Interés' not in x]
                words=[x for x in words if 'OFF' not in x]
                words=[x for x in words if 'Envío' not in x]
                words=[x for x in words if 'Ahorrá' not in x]
                words=[x for x in words if '%' not in x]
                words=[x for x in words if 'Hasta' not in x]
                words=[x for x in words if 'GRATIS' not in x]
                if len(words)<3:
                    words.append('null')
                words.append(date_time)
                words.append(store)
                print(words)
                data.append(words)


            try:
                driver.execute_script("window.scrollBy(0,2300)")
                nextpage = driver.find_element(By.XPATH, '//*[@id="botones"]/button[{}]'.format(page))
                time.sleep(2)
                nextpage.click()
                time.sleep(2)
            
            except:
                print('data scrapeada')
                break
        
        print('megatone scrapeado')
        return data

    elif store=='coppel':
        itemlist = []
        data=[]
        for i in range(3):
            page=i+1
            driver.execute_script("window.scrollBy(0,50)")
            time.sleep(2)

            items = driver.find_elements(By.CSS_SELECTOR,'.vtex-product-summary-2-x-element')

            for i in range(len(items)):
                itemlist.append(items[i].text)
            itemlist = list(filter(None,itemlist))
            arrayitems=[]
            for i in range (len(itemlist)):
        
                words=str(itemlist[i]).split('\n')
                words=[x for x in words if 'Sin Interés' not in x]
                words=[x for x in words if 'OFF' not in x]
                words=[x for x in words if 'Envío' not in x]
                words=[x for x in words if 'Ahorrá' not in x]
                words=[x for x in words if '%' not in x]
                words=[x for x in words if 'Hasta' not in x]
                if len(words)<3:
                    words.append('null')
                words.append(date_time)
                words.append(store)
                print(words)
                data.append(words)

            

            try:
                driver.get(url+'&page={}'.format(page+1))
                time.sleep(5)            
            except:
                print('data scrapeada')
                break
        print('coppel scrapeado')
        return data
    
    elif store=='fravega':

        data=[]
        time.sleep(2)
        close = driver.find_element(By.XPATH, '//*[@id="modal"]/div[1]/button')
        close.click()
        for i in range(10):
            page=i+1
            items=[]
            for i in range(63):
                try:
                    item = driver.find_elements(By.XPATH,'//*[@id="__next"]/div[2]/div[1]/div[3]/div[5]/ul/li[{}]'.format(i+1))
                    items.append(item)


                except:
                    print('no se encontro el item')
                    break
                    
            listael=[]
            for i in items:
                for z in i:
                    v=z.text
                    listael.append(v)

            prods=[]
            for i in listael:
                words=i.split('''\n''')
                words=[x for x in words if '¡Retiralo YA!' not in x]
                words=[x for x in words if 'Comparar' not in x]
                words=[x for x in words if 'Llega GRATIS en 48hs' not in x]
                words=[x for x in words if 'Llega en 48hs' not in x]
                words=[x for x in words if 'Retiro en 48hs' not in x]
                words=[x for x in words if 'Envío GRATIS' not in x]
                words=[x for x in words if 'Llega GRATIS mañana' not in x]
                if len(words)<3:
                    words.append('null')
                words.append(date_time)
                words.append(store)
                print(words)
                prods.append(words)


            for i in prods:
                item=[]
                for z in i:
                    try:
                        int(z)/2
                    except:
                        item.append(z)
                data.append(item)

            try:
                driver.get(url+'&page={}'.format(page+1))
                time.sleep(5)
            
            except:
                print('data scrapeada')
                break
            try:
                time.sleep(2)
                close = driver.find_element(By.XPATH, '//*[@id="modal"]/div[1]/button')
                close.click()
            except:
                print('no hay overlay')

        print('fravega scrapeado')
        return data


 

for i in range(len(productos)):
    pars=(productos.loc[i,:].values.flatten().tolist())
    if pars[1]=="megatone":
        scrappage='https://www.megatone.net/search/{}/'.format(pars[0])
        megatone_data=scrap_page(scrappage,art=pars[0],store='megatone')

    elif pars[1]=="coppel":
        scrappage='https://www.coppel.com.ar/{}?_q={}&map=ft'.format(pars[0],pars[0])
        coppel_data=scrap_page(scrappage,art=pars[0],store='coppel')
        
    elif pars[1]=="fravega":
        scrappage='https://www.fravega.com/l/?keyword={}'.format(pars[0]).replace(' ','+')
        fravega_data=scrap_page(scrappage,art=pars[0],store='fravega')





se busca en https://www.megatone.net/search/Escritorio Notebook 3045-Coe O Tables/ en fecha y hora: 2023-07-31 20:59:35.889485
['Escritorio Notebook 3045-Coe O Tables', '$47,299', '$38,999', '2023-07-31 20:59:35.889485', 'megatone']
data scrapeada
megatone scrapeado
se busca en https://www.fravega.com/l/?keyword=Mouse+Genius+Nx+7010+Blueeye+White/red+(new+Package)+(8629)+Genius en fecha y hora: 2023-07-31 20:59:35.889485
['Mouse Genius Nx 7010 Blueeye White/blue (new Package) (8612) Genius', '$18.370', '9', '$16.700', '2023-07-31 20:59:35.889485', 'fravega']
['Mouse Genius Nx-7010 Blueeye Magenta (new Package) (8636) Genius', '$18.370', '9', '$16.700', '2023-07-31 20:59:35.889485', 'fravega']
['Mouse Genius Nx 7010 Blueeye White/red (new Package) (8629) Genius', '$18.388', '9', '$16.717', '2023-07-31 20:59:35.889485', 'fravega']
fravega scrapeado
se busca en https://www.megatone.net/search/Notebook I7 15Itl6 Ip3 8Gb256ssd W1 Lenovo/ en fecha y hora: 2023-07-31 20:59:35.889485
['Noteboo

In [4]:
data=[]
for i in coppel_data:
    data.append(i)

for i in megatone_data:
    data.append(i)

for i in fravega_data:
    data.append(i)

print(data)

[['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB 4g Lte SM-A235M', '$ 122.999', 'null', '2023-07-31 20:59:35.889485', 'coppel'], ['Celular Samsung A23 Celeste 4GB 128GB

In [5]:
import csv

fields = ['producto','precio','precio_rebaja','date_time','tienda']

with open('data.csv','w',newline='') as f:
    writer = csv.writer(f,delimiter='#')
    writer.writerow(fields)
    writer.writerows(data)

In [6]:
# ahora hay que crear un entorno en airflow y probarlo